# EXTRACT 

In [19]:
import awswrangler as awr
import logging
import pandas as pd
import os

logging.basicConfig(
    level=logging.INFO,  # Exibe mensagens a partir de INFO
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler()  # Garante logs no console
    ]
)

logging.info('\n ----------------------------------------------------------------------------------')
logging.info('\n Executando Rotina: Movimentação de Placas')

class Extract:

    def __init__(self):
        self.path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes"

    def extract_all_ativacoes(self):

        try:
            dir_query = os.path.join(self.path, 'sql', 'all_boards_ATIVOS.sql')
            with open(dir_query, 'r') as file:
                query = file.read()
            df_ativacoes = awr.athena.read_sql_query(query, database='silver')
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de ativações extraída com sucesso!')
            return df_ativacoes
        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair a consulta de ativações: {e}')
            return None

    def extract_all_cancelamentos(self):

        try:
            dir_query = os.path.join(self.path, 'sql', 'all_boards_CANCELADOS.sql')
            with open(dir_query, 'r') as file:
                query = file.read()
            df_cancelamentos = awr.athena.read_sql_query(query, database='silver')
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de cancelamentos extraída com sucesso!')
            return df_cancelamentos
        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair a consulta de cancelamentos: {e}')
            return None

extract = Extract()
df_ativacoes = extract.extract_all_ativacoes()
df_cancelamentos = extract.extract_all_cancelamentos()



2025-08-28 17:43:50,228 - INFO - 
 ----------------------------------------------------------------------------------
2025-08-28 17:43:50,240 - INFO - 
 Executando Rotina: Movimentação de Placas
2025-08-28 17:43:50,335 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:43:51,351 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:43:52,901 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:43:54,335 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:43:55,331 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:43:57,289 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:43:59,266 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:44:01,867 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:44:03,8

# TRANSFORM

In [20]:
# IMPORTANDO MÓDULOS E PACOTES
import pandas as pd
import numpy as np
import datetime as dt
import logging


class Transform:
        
    def __init__(self) -> None:
        pass

    def transforming_files(self):

#DEFININDO DATAFRAMES VAZIOS 
        try:

            df_final_ativacoes = pd.DataFrame()
            df_final_cancelamentos = pd.DataFrame()

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('Falha ao definir dataframes.')

#TRANSFORMANDO DF_ATIV E SEGMENTANDO POR EMPRESA
        try:
            extract = Extract()
            df_final_cancelamentos = extract.extract_all_cancelamentos()
            df_ativ_all_boards = extract.extract_all_ativacoes()

            df_ativ_all_boards['data_ativacao_beneficio'] = pd.to_datetime(df_ativ_all_boards['data_ativacao_beneficio']).dt.date
                        
            df_ativ_all_boards['beneficio'] = df_ativ_all_boards['beneficio'].replace('REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO', 'CASCO (VEÍCULO)').replace('REPARAÇÃO OU REPOSIÇÃO DO (SEMI)REBOQUE', 'CASCO (R/SR)').replace('REPARAÇÃO OU REPOSIÇÃO DO COMPLEMENTO', 'CASCO (COMPLEMENTO)')
            
            df_ativ_viavante = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Viavante']
            df_ativ_stcoop = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Stcoop']
            df_ativ_segtruck = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Segtruck']
            df_ativ_tag = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Tag']

            df_final_cancelamentos = df_final_cancelamentos

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')  
            logging.info(f'Falha ao realizar a segmentação dos dataframes: {e}')

# SELECIONANDO APENAS AS ATIVAÇÕES CORRESPONDENTES AOS BENEFICIOS 'CASCO' / 'TERCEIRO' POR UM REGEX PADRÃO
        try:
            ids_beneficios_segtruck = [2, 3, 4, 7, 24, 25, 26, 29]
            ids_beneficios_stcoop = [24, 25, 26, 29]
            ids_beneficios_viavante = [40, 41, 42, 45]
            ids_beneficios_tag = [2, 3, 4, 7, 24, 25, 26, 29, 34, 35, 36, 37, 38, 39]

            df_ativ_viavante = df_ativ_viavante.loc[df_ativ_viavante['id_beneficio'].isin(ids_beneficios_viavante)]
            df_ativ_stcoop = df_ativ_stcoop[df_ativ_stcoop['id_beneficio'].isin(ids_beneficios_stcoop)]
            df_ativ_segtruck = df_ativ_segtruck.loc[df_ativ_segtruck['id_beneficio'].isin(ids_beneficios_segtruck)]
            df_ativ_tag = df_ativ_tag.loc[df_ativ_tag['id_beneficio'].isin(ids_beneficios_tag)]

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')  
            logging.info(f'Falha ao padronizar nomenclaturas referente aos beneficios pré-estabelecidos: {e}')

# CONCATENANDO E CRIANDO COLUNA DE MIGRAÇÃO (MIGRATION_FROM) 
        try:

            df_final_ativacoes = pd.concat([df_ativ_viavante, df_ativ_stcoop, df_ativ_segtruck, df_ativ_tag])

            if not df_final_ativacoes.empty:
                df_final_ativacoes['migration_from'] = np.nan

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha na criação da coluna de migração e concatenação de dataframes: {e}')

    # DEFININDO COLUNAS QUE SERÃO UTILIZADAS NO DATAFRAME FINAL
        try:
           
            df_final_ativacoes = df_final_ativacoes[[
                'placa', 'chassi', 'id_placa', 'id_veiculo', 'id_carroceria', 'matricula', 'conjunto', 'unidade', 'consultor', 'status_beneficio', 
                'cliente', 'data_registro', 'data_ativacao_beneficio', 'suporte', 'data_filtro', 'empresa', 'migration_from'
            ]]

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Processo de seleção de colunas realizado com sucesso!')

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao definir colunas: {e}')

    # RETIRANDO DUPLICATAS
        try:
            df_final_ativacoes = df_final_ativacoes.drop_duplicates(subset=['chassi'])

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Duplicatas retiradas com sucesso.')

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao retirar duplicatas. Revise o código: {e}')

# TRATANDO DADOS NULOS NOS DATAFRAMES
        try:
            df_final_ativacoes['placa'] = df_final_ativacoes['placa'].fillna('SEM-PLACA')
            df_final_ativacoes['chassi'] = df_final_ativacoes['chassi'].fillna('NULL')
            df_final_ativacoes['id_placa'] = df_final_ativacoes['id_placa'].fillna(0)
            df_final_ativacoes['id_veiculo'] = df_final_ativacoes['id_veiculo'].fillna(0)
            df_final_ativacoes['id_carroceria'] = df_final_ativacoes['id_carroceria'].fillna(0)
            df_final_ativacoes['matricula'] = df_final_ativacoes['matricula'].fillna(0)
            df_final_ativacoes['conjunto'] = df_final_ativacoes['conjunto'].fillna(0)
            df_final_ativacoes['unidade'] = df_final_ativacoes['unidade'].fillna('NULL')
            df_final_ativacoes['consultor'] = df_final_ativacoes['consultor'].fillna('NULL')
            df_final_ativacoes['status_beneficio'] = df_final_ativacoes['status_beneficio'].fillna('NULL')
            df_final_ativacoes['cliente'] = df_final_ativacoes['cliente'].fillna('NULL')
            df_final_ativacoes['data_registro'] = df_final_ativacoes['data_registro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['data_ativacao_beneficio'] = df_final_ativacoes['data_ativacao_beneficio'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['suporte'] = df_final_ativacoes['suporte'].fillna('NULL')
            df_final_ativacoes['data_filtro'] = df_final_ativacoes['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['empresa'] = df_final_ativacoes['empresa'].fillna('NULL')
            df_final_ativacoes['migration_from'] = df_final_ativacoes['migration_from'].fillna('NULL')

            df_final_cancelamentos['placa'] = df_final_cancelamentos['placa'].fillna('SEM-PLACA')
            df_final_cancelamentos['chassi'] = df_final_cancelamentos['chassi'].fillna('NULL')
            df_final_cancelamentos['id_placa'] = df_final_cancelamentos['id_placa'].fillna(0)
            df_final_cancelamentos['id_veiculo'] = df_final_cancelamentos['id_veiculo'].fillna(0)
            df_final_cancelamentos['id_carroceria'] = df_final_cancelamentos['id_carroceria'].fillna(0)
            df_final_cancelamentos['matricula'] = df_final_cancelamentos['matricula'].fillna(0)
            df_final_cancelamentos['conjunto'] = df_final_cancelamentos['conjunto'].fillna(0)
            df_final_cancelamentos['unidade'] = df_final_cancelamentos['unidade'].fillna('NULL')
            df_final_cancelamentos['status'] = df_final_cancelamentos['status'].fillna('NULL')
            df_final_cancelamentos['cliente'] = df_final_cancelamentos['cliente'].fillna('NULL')
            df_final_cancelamentos['data'] = df_final_cancelamentos['data'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos['data_cancelamento'] = df_final_cancelamentos['data_cancelamento'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos['usuario_cancelamento'] = df_final_cancelamentos['usuario_cancelamento'].fillna('NULL')
            df_final_cancelamentos['data_filtro'] = df_final_cancelamentos['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos['empresa'] = df_final_cancelamentos['empresa'].fillna('NULL')
            df_final_cancelamentos['migracao'] = df_final_cancelamentos['migracao'].fillna('NULL')
            df_final_cancelamentos['renegociacao'] = df_final_cancelamentos['renegociacao'].fillna('NULL')
            df_final_cancelamentos['data_substituicao'] = df_final_cancelamentos['data_substituicao'].fillna(pd.Timestamp('1900-01-01'))

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Processo de Transformacao de Dados concluido com sucesso!')

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao realizar tratamento de dados: {e}')
     
        return df_final_ativacoes, df_final_cancelamentos
    
transform = Transform()
df_final_ativacoes, df_final_cancelamentos = transform.transforming_files()

        

2025-08-28 17:45:11,744 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:45:13,482 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:45:15,323 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:45:16,822 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:45:17,879 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:45:19,813 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:45:21,792 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:45:23,775 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:45:25,730 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:45:27,710 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-28 17:45:29,

In [21]:
df_final_ativacoes_ativos = df_final_ativacoes[df_final_ativacoes['status_beneficio']=='ATIVO']
df_final_ativacoes_ativos.shape[0]

32932

In [22]:
df_final_ativacoes_ativos.head()

,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,consultor,status_beneficio,cliente,data_registro,data_ativacao_beneficio,suporte,data_filtro,empresa,migration_from
32645,TAS2G52,94BF1544RSV007748,18522,0,18522,1208,11499,MF - MARTUCCI SOLUCOES,Amilton Donizete Martucci,ATIVO,VGM TRANSPORTES LTDA.,2025-03-06,2025-03-12,VICTOR GABRIEL GOMES PICÃO,2025-08-28,Viavante,NULL
32647,NVU5E18,9BSG6X200B3674145,183606,183606,0,1214,18922,UNIDADE CASCAVEL,Thiago Rosa De Figueiredo,ATIVO,JAMIL FERNANDO AHMAD LTDA,2025-06-23,2025-07-05,João Paulo Ribeiro Pinto,2025-08-28,Viavante,NULL
32649,AWF8855,9BVAG20D6EE816037,183603,183603,0,1215,18923,UNIDADE LONDRINA,Samuel Victor Martins Sousa,ATIVO,JOAO PADILHA NETO,2025-06-23,2025-07-05,Lucas Belmiro Mendes Santos,2025-08-28,Viavante,NULL
32651,SEY5F07,9539K8TJ4RR200808,14255946,14255946,0,1215,5869,UNIDADE LONDRINA,Carolline Munhoz Guerra,ATIVO,JOAO PADILHA NETO,2024-11-22,2024-11-22,João Paulo Ribeiro Pinto,2025-08-28,Viavante,NULL
32656,RBS8I50,9BVRG40D0ME893999,183611,183611,0,1224,20109,MF - R A SERVICOS & CONSULTORIA LTDA,Rafaella Brenner,ATIVO,LUIZ CARLOS NUNES DE SOUZA,2025-07-08,2025-07-09,Lucas Lobo Didur,2025-08-28,Viavante,NULL


# LOAD

In [23]:
# IMPORTANDO MÓDULOS E PACOTES
import logging
import openpyxl
import os
import pandas as pd
import logging

today = pd.Timestamp.today().date()
yesterday = today - pd.Timedelta(days=1)
dbf_yesterday = today - pd.Timedelta(days=2)


file_path = rf"C:\Users\raphael.almeida\Documents\Processos\placas_acompanhamento\template\placas_movimentacoes_{today}.xlsx"

destination_dir = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas"
destination_path = os.path.join(destination_dir, os.path.basename(file_path))

destination_dir2 = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\bkp_activation"
destination_path2 = os.path.join(destination_dir2, os.path.basename(file_path))

with pd.ExcelWriter(destination_path, engine='openpyxl') as writer:
    df_final_ativacoes.to_excel(writer, index=False, sheet_name='ATIVAÇÕES')
    df_final_cancelamentos.to_excel(writer, index=False, sheet_name='CANCELAMENTOS')

with pd.ExcelWriter(destination_path2, engine='openpyxl') as writer:
    df_final_ativacoes.to_excel(writer, index=False, sheet_name='ATIVAÇÕES')
    df_final_cancelamentos.to_excel(writer, index=False, sheet_name='CANCELAMENTOS')

logging.info('\n ----------------------------------------------------------------------------------')
logging.info('\n Processo de Carregamento de Dados concluido com sucesso!')




    


2025-08-28 17:48:16,453 - INFO - 
 ----------------------------------------------------------------------------------
2025-08-28 17:48:16,455 - INFO - 
 Processo de Carregamento de Dados concluido com sucesso!


In [24]:
# import logging
# import openpyxl
# import os
# import pandas as pd
# import logging

# today = pd.Timestamp.today().date()
# yesterday = today - pd.Timedelta(days=1)
# dbf_yesterday = today - pd.Timedelta(days=2)

# file_path = rf"C:\Users\raphael.almeida\Documents\Processos\placas_acompanhamento\template\placas_movimentacoes_{today}.xlsx"

# destination_dir = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas"
# destination_path = os.path.join(destination_dir, os.path.basename(file_path))

# destination_dir2 = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\bkp_activation"
# destination_path2 = os.path.join(destination_dir2, os.path.basename(file_path))

# with pd.ExcelWriter(destination_path, engine='openpyxl') as writer:
#     df_final_cancelamentos.to_excel(writer, index=False, sheet_name='CANCELAMENTOS')

# with pd.ExcelWriter(destination_path2, engine='openpyxl') as writer:
#     df_final_cancelamentos.to_excel(writer, index=False, sheet_name='CANCELAMENTOS')